In [ ]:
import os
import sys
import pandas as pd
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm

import openai

import config



In [ ]:
TEXTS_DIR = './input_texts/'

### get all the documents

In [ ]:
files = os.listdir(TEXTS_DIR)
files = [x for x in files if x.endswith('.txt')]
print (f'{len(files)} documents found.')
print (files)

In [ ]:
loaders = [
    TextLoader(os.path.join(TEXTS_DIR, f)) for f in files
]

docs = []
for loader in tqdm(loaders):
    docs.extend(loader.load())

In [ ]:
from langchain.document_loaders import UnstructuredHTMLLoader
files = os.listdir(TEXTS_DIR)
files = [x for x in files if x.endswith('.html')]
print (files)
loaders = [
    UnstructuredHTMLLoader(os.path.join(TEXTS_DIR, f)) for f in files

]

htmls = []
for loader in tqdm(loaders):
    htmls.extend(loader.load())



### Split texts

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 5000,
    chunk_overlap = 200,
    separators=["\n\n", "\n", " ", ""]
)

splits = text_splitter.split_documents(docs)
len(splits)

Now html data

In [ ]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)
#print(RecursiveCharacterTextSplitter.get_separators_for_language(Language.HTML))

html_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.HTML, chunk_size=5000, chunk_overlap=200
)

html_splits = html_splitter.split_documents(htmls)
len(html_splits)


In [ ]:
len(splits + html_splits)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    openai_api_key=config.OPENAI_KEY,
    model = 'text-embedding-ada-002'
    )

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = './chroma/'
vectordb = Chroma.from_documents(
    documents=splits + html_splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
vectordb.persist()